In [ ]:
import pandas as pd
import numpy as np
import torch
from scipy.spatial import distance_matrix
import numpy as np
import tensorflow
import spektral
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from spektral.data import Dataset, BatchLoader, Graph
from spektral.transforms.normalize_adj import NormalizeAdj
from tensorflow.keras.layers import Dropout
from spektral.layers import GCNConv, GlobalSumPool

In [3]:

df = pd.read_csv('train.csv', header=[0, 1])
meta_df = pd.read_csv("metaData.csv")
#get target feature out of df
target = df[['Timestamp',"target_feature"]].rename(columns={"Unnamed: 1_level_1":"","Unnamed: 53_level_1":""})
target.columns = [''.join(col).strip() for col in target.columns] 
target.drop(target.iloc[0].name, inplace=True)
target["Timestamp"] = pd.to_datetime(target["Timestamp"])
df.head()

C:\Users\conny\AppData\Local\Temp\ipykernel_17048\1690104781.py:1: DtypeWarning: Columns (0,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('train.csv', header=[0, 1])


Unnamed: 0_level_0            Timestamp Wind speed (m/s)             \
             Turbine   Unnamed: 1_level_1       Kelmarsh 2 Kelmarsh 3   
0                 id                  NaN              NaN        NaN   
1                  0  2017-10-01 00:00:00         9.112875   9.197829   
2                  1  2017-10-01 00:10:00         8.319863   9.350296   
3                  2  2017-10-01 00:20:00         8.799176   8.510402   
4                  3  2017-10-01 00:30:00         8.392921   8.825861   

                                   Wind speed, Standard deviation (m/s)  \
  Kelmarsh 4 Kelmarsh 5 Kelmarsh 6                           Kelmarsh 2   
0        NaN        NaN        NaN                                  NaN   
1   8.795919   8.749384   7.684954                             1.104222   
2   8.762059   8.652318   7.421576                             1.207655   
3   9.533566   8.166586   7.797040                             1.124029   
4   8.990985   7.877046   7.750090                             1.219647   

                         ... Generator RPM (RPM)                            \
  Kelmarsh 3 Kelmarsh 4  ...          Kelmarsh 4   Kelmarsh 5   Kelmarsh 6   
0        NaN        NaN  ...                 NaN          NaN          NaN   
1   1.096619   0.999588  ...         1768.215088  1733.296875  1603.749390   
2   0.916610   0.956899  ...         1770.806885  1737.218262  1572.769409   
3   0.992841   1.343180  ...         1766.002319  1698.699219  1655.049316   
4   1.218454   0.970566  ...         1783.569824  1661.264526  1678.571777   

  Blade angle (pitch position) A (°)                                   \
                          Kelmarsh 2 Kelmarsh 3 Kelmarsh 4 Kelmarsh 5   
0                                NaN        NaN        NaN        NaN   
1                           0.183487   0.135490   0.001053     0.0000   
2                           0.051998   0.086999   0.095998     0.0605   
3                           0.168988   0.056997   0.087998     0.0090   
4                           0.084492   0.133995   0.017500     0.0000   

                        training      target_feature  
  Kelmarsh 6 Unnamed: 52_level_1 Unnamed: 53_level_1  
0        NaN                 NaN                 NaN  
1      0.000                True            8.602844  
2      0.000                True            8.125226  
3      0.016                True            7.551986  
4      0.000                True            7.912114  

[5 rows x 54 columns]

In [4]:
df.columns = ['_'.join(col).strip() for col in df.columns]  # Flatten multi-index
df.rename(columns={"Timestamp_Unnamed: 1_level_1": "Timestamp"}, inplace=True)  # Rename timestamp
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

In [5]:
df_long = df.melt(id_vars=["Timestamp"], 
                  var_name="Variable_Turbine", 
                  value_name="Value")

# Split 'Variable_Turbine' into 'Variable' and 'Turbine'
df_long[["Variable", "Turbine"]] = df_long["Variable_Turbine"].str.rsplit("_", n=1, expand=True)
df_long.drop(columns=["Variable_Turbine"], inplace=True)
df_long=df_long[df_long["Turbine"].str.contains("Kelmarsh")]
df_long.head()  # View the reshaped dataframedf

,Timestamp,Value,Variable,Turbine
130609,NaT,NaN,Wind speed (m/s),Kelmarsh 2
130610,2017-10-01 00:00:00,9.112875,Wind speed (m/s),Kelmarsh 2
130611,2017-10-01 00:10:00,8.319863,Wind speed (m/s),Kelmarsh 2
130612,2017-10-01 00:20:00,8.799176,Wind speed (m/s),Kelmarsh 2
130613,2017-10-01 00:30:00,8.392921,Wind speed (m/s),Kelmarsh 2


In [6]:
df_wide = df_long.pivot(index=['Timestamp', 'Turbine'], columns='Variable', values='Value').reset_index()
df_wide = df_wide[df_wide["Timestamp"].notna()]

In [7]:
meta_df["total_hub_height"] = meta_df["Hub Height (m)"]  + meta_df["Elevation (m)"] 

In [8]:
#df_wide_meta = df_wide.merge(meta_df, left_on="Turbine",right_on="Title")

In [9]:
df_final = df_wide.merge(target, on="Timestamp")

In [10]:
df_final

,Timestamp,Turbine,Blade angle (pitch position) A (°),Generator RPM (RPM),Nacelle ambient temperature (°C),Nacelle position (°),"Nacelle position, Standard deviation (°)",Power (kW),Vane position 1+2 (°),Wind direction (°),Wind speed (m/s),"Wind speed, Standard deviation (m/s)",target_feature
0,2017-10-01 00:00:00,Kelmarsh 2,0.183487,1793.296997,12.9,171.46228,0.0,1484.768311,-5.900865,166.338806,9.112875,1.104222,8.602844
1,2017-10-01 00:00:00,Kelmarsh 3,0.13549,1780.72937,12.95,174.632751,0.0,1386.177734,-4.050556,171.097855,9.197829,1.096619,8.602844
2,2017-10-01 00:00:00,Kelmarsh 4,0.001053,1768.215088,13.3,164.43544,0.406891,1277.073242,1.26274,166.757828,8.795919,0.999588,8.602844
3,2017-10-01 00:00:00,Kelmarsh 5,0.0,1733.296875,12.4975,159.976288,0.0,1129.36731,6.251606,167.49202,8.749384,1.192191,8.602844
4,2017-10-01 00:00:00,Kelmarsh 6,0.0,1603.74939,13.83,170.339966,0.0,850.010681,-5.537934,167.577591,7.684954,1.290311,8.602844
...,...,...,...,...,...,...,...,...,...,...,...,...,...
653035,2021-06-16 23:50:00,Kelmarsh 2,NaN,540.960022,NaN,258.320007,35.720001,38.959999,NaN,299.589996,3.76,0.52,3.940000
653036,2021-06-16 23:50:00,Kelmarsh 3,NaN,290.48999,NaN,229.720001,43.439999,14.97,NaN,330.920013,3.68,0.69,3.940000
653037,2021-06-16 23:50:00,Kelmarsh 4,NaN,364.140015,NaN,245.600006,34.439999,22.76,NaN,307.73999,3.53,0.66,3.940000
653038,2021-06-16 23:50:00,Kelmarsh 5,NaN,169.559998,NaN,210.320007,43.57,-2.14,NaN,318.98999,3.35,0.64,3.940000


In [ ]:
def get_turbine_distances(meta_df):
    """
    Compute pairwise distances and height differences between turbines.

    Args:
        meta_df (pd.DataFrame): DataFrame containing 'Latitude', 'Longitude', and 'total_hub_height'.
        threshold (float): Maximum distance (meters) to consider an edge.

    Returns:
        dict: {("Turbine A", "Turbine B"): (distance, height_difference)}
        torch.Tensor: edge_index for PyG
        torch.Tensor: edge_attr containing (distance, height_difference)
    """
    coords = meta_df[['Latitude', 'Longitude']].values
    
    # Calculate pairwise distances
    dist_matrix = distance_matrix(coords, coords)
    
    return dist_matrix

In [12]:
turbine_distances, edge_index, edge_attr = get_turbine_distances(meta_df)

In [ ]:
################################################################################
# Config
################################################################################
learning_rate = 1e-2  # Learning rate
epochs = 100  # Number of training epochs
es_patience = 10  # Patience for early stopping
batch_size = 21  # Batch size


In [ ]:
################################################################################
# Load data
################################################################################

class GraphDataset(Dataset):
    def __init__(self, n_samples, df, n_colors=120, **kwargs):
        self.n_samples = n_samples
        self.df = df  
        self.n_colors = n_colors  
        super().__init__(**kwargs)

    def read(self):
        output = []
        for i in range(self.n_samples):
            # Node features
            iter_x = self.df["train_x"][i].copy()
            x = np.array(iter_x).reshape(5,#nfeatures)

            # Edges
            iter_a =  self.df["train_a"][i].copy()
            a = np.array(iter_a).reshape(5,5)

            # 
            y = np.zeros((120,))
            y_index = int(19+self.df["y"][i])
            y[y_index:] = 1
           
            output.append(Graph(x=x, a=a, y=y))
            print(i)
        return(output)

In [ ]:

# Train/valid/test split
len_train = len(local_train["train_x"])
len_val = len(local_val["train_x"])
len_test = len(local_test["train_x"])
len_all = len(local_all["train_x"])
len_loo_vis= len(local_loo_vis["train_x"])
data_tr = GraphDataset(n_samples = len_train, df = local_train, transforms=NormalizeAdj())
data_va = GraphDataset(n_samples = len_val, df = local_val, transforms=NormalizeAdj())
data_te = GraphDataset(n_samples = len_test, df=local_test, transforms=NormalizeAdj())
data_all = GraphDataset(n_samples = len_all, df=local_all, transforms=NormalizeAdj())
data_loo_vis = GraphDataset(n_samples = len_loo_vis, df=local_loo_vis, transforms=NormalizeAdj())
# Data loaders
loader_tr = BatchLoader(data_tr, batch_size=batch_size, epochs=epochs)
loader_va = BatchLoader(data_va, batch_size=batch_size)
loader_te = BatchLoader(data_te, batch_size=batch_size)
loader_all = BatchLoader(data_all, batch_size=batch_size)
loader_loo_vis = BatchLoader(data_loo_vis, batch_size=batch_size)

In [ ]:
################################################################################
# Build model
################################################################################
class BDB22GNN(Model):

    def __init__(self, n_hidden, n_labels):
        super().__init__()
        self.graph_conv = GCNConv(n_hidden)
        self.pool = GlobalSumPool()
        self.dropout = Dropout(0.5)
        self.dense = Dense(n_labels, 'sigmoid')

    def call(self, inputs):
        out = self.graph_conv(inputs)
        out = self.dropout(out)
        out = self.pool(out)
        out = self.dense(out)

        return out

In [ ]:

model = BDB22GNN(21, 120)
model.compile('adam', "mean_absolute_error")

model.fit(loader_tr.load(), validation_data= loader_va.load(), steps_per_epoch=loader_tr.steps_per_epoch,
    validation_steps=loader_va.steps_per_epoch, epochs=100)

test_loss = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)

print('Test loss: {}'.format(test_loss))
  

predictions = model.predict(loo_loader.load(), steps =loo_loader.steps_per_epoch)

